In [1]:
import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer
import langid
import re
import torch
from tqdm import tqdm
import random
import os
import time


2025-04-21 07:25:52.583137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745220352.891995      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745220352.973316      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
# === Настройки ===
csv_path = "/kaggle/input/nicereviews2/NICE_reviews1.csv"
output_path = "/kaggle/working/summaries_NICE2.csv"
START_FROM_GAME = "Mirror's Edge™"  # <- укажи нужную игру
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_reviews = 1000
batch_size = 30
PROCESS_START = 0     # Если хочешь разбить между GPU - можно указать тут срез
PROCESS_END = None    # Например: PROCESS_START = 0, PROCESS_END = 50


In [5]:
# === Загрузка моделей ===
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

translation_model_name = "Helsinki-NLP/opus-mt-en-ru"
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translation_model = MarianMTModel.from_pretrained(translation_model_name).to(device)

print(f"Модели успешно загружены. Используется устройство: {device}")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Модели успешно загружены. Используется устройство: cuda


model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

In [9]:
def is_valid_review(text):
    if not text or len(text.strip()) < 20:
        return False
    if len(re.findall(r'\w+', text)) < 5:
        return False
    if re.fullmatch(r"[\W\d_]+", text):
        return False
    if re.search(r'---\s*\w+\s*---', text):
        return False
    if re.search(r'\b(ong|peak|10/10|best game|love it|lol|omg|hilarious|funny|memes)\b', text.lower()) and len(text.strip()) < 50:
        return False
    return True

def clean_summary(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'---.*?---', '', text)
    text = re.sub(r'[–—]{2,}', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def split_reviews_by_lang(reviews, allowed_langs=('en',)):
    lang_groups = defaultdict(list)
    for review in reviews:
        if is_valid_review(review):
            lang, _ = langid.classify(review)
            if lang in allowed_langs:
                lang_groups[lang].append(review.strip())
    return lang_groups

def summarize_text(text, max_input_len=2048, max_output_len=300):
    prompt = (
        "Summarize the following user reviews of a video game in a coherent paragraph. "
        "Highlight the gameplay, atmosphere, performance, and overall experience. "
        "Avoid repetition, usernames, or jokes:\n\n" + text
    )
    inputs = tokenizer(prompt, return_tensors="pt", max_length=max_input_len, truncation=True).to(device)
    summary_ids = model.generate(
        **inputs,
        max_length=max_output_len,
        min_length=100,
        do_sample=True,
        temperature=0.7,
        top_k=30,
        top_p=0.9,
        repetition_penalty=1.5,
        num_beams=1,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_reviews(reviews, batch_size=batch_size):
    partial_summaries = []
    for i in tqdm(range(0, len(reviews), batch_size), desc="Обработка отзывов"):
        batch = reviews[i:i+batch_size]
        text = " ".join(batch)
        summary = summarize_text(text)
        partial_summaries.append(summary)
    combined_text = " ".join(partial_summaries)
    return summarize_text(combined_text, max_input_len=2048, max_output_len=300)

def sample_reviews(reviews, n=max_reviews):
    return random.sample(reviews, n) if len(reviews) > n else reviews

def translate_to_russian(text):
    inputs = translation_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    translated = translation_model.generate(**inputs, max_length=512)
    return translation_tokenizer.decode(translated[0], skip_special_tokens=True)


In [10]:
# === Подготовка данных ===
df = pd.read_csv(csv_path, sep=';')
all_games = df["game_title"].dropna().unique().tolist()

# Учитываем диапазон PROCESS_START/PROCESS_END
all_games = all_games[PROCESS_START:PROCESS_END]

# Продолжение с определённой игры
if START_FROM_GAME in all_games:
    start_index = all_games.index(START_FROM_GAME)
    games = all_games[start_index:]
else:
    games = all_games

# Загрузка уже обработанных результатов
if os.path.exists(output_path):
    saved_df = pd.read_csv(output_path)
    processed_games = set(saved_df["game_title"].unique())
else:
    saved_df = pd.DataFrame(columns=["game_title", "lang", "summary_ru", "summary_en"])
    processed_games = set()

# Исключение уже обработанных игр
games = [g for g in games if g not in processed_games]

# === Основной цикл ===
for game_name in tqdm(games, desc="Обработка игр"):
    start_time = time.time()

    game_reviews = df[df["game_title"] == game_name]["review_text"].dropna().tolist()
    lang_groups = split_reviews_by_lang(game_reviews)

    if not lang_groups:
        print(f"⏭️ Пропуск {game_name} — нет подходящих отзывов")
        continue

    for lang, reviews in lang_groups.items():
        if len(reviews) < 10:
            print(f"❌ {game_name} ({lang}): слишком мало отзывов")
            continue

        reviews = sample_reviews(reviews)
        print(f"🚀 Суммаризация: {game_name} | Язык: {lang} | Отзывов: {len(reviews)}")

        try:
            summary = summarize_reviews(reviews)
            summary = clean_summary(summary)

            if lang == "en":
                summary_en = summary
                summary_ru = translate_to_russian(summary)
            else:
                summary_ru = summary
                summary_en = translate_to_russian(summary)

            # Сохранение в CSV
            new_row = pd.DataFrame([{
                "game_title": game_name,
                "lang": lang,
                "summary_ru": summary_ru,
                "summary_en": summary_en
            }])
            new_row.to_csv(output_path, mode="a", index=False, header=not os.path.exists(output_path))

            end_time = time.time()
            elapsed = int(end_time - start_time)
            mins, secs = divmod(elapsed, 60)
            print(f"✅ {game_name} ({lang}) — {mins} мин {secs} сек")

        except Exception as e:
            print(f"⚠️ Ошибка при обработке {game_name} ({lang}): {e}")


Обработка игр:   0%|          | 0/9 [00:00<?, ?it/s]

🚀 Суммаризация: Mirror's Edge™ | Язык: en | Отзывов: 1000



Обработка игр:  11%|█         | 1/9 [03:27<27:37, 207.22s/it]

✅ Mirror's Edge™ (en) — 3 мин 27 сек
🚀 Суммаризация: Detroit: Become Human | Язык: en | Отзывов: 1000



Обработка игр:  22%|██▏       | 2/9 [06:59<24:29, 209.97s/it]

...

✅ Raft (en) — 3 мин 17 сек
🚀 Суммаризация: Little Nightmares | Язык: en | Отзывов: 1000



Обработка игр:  89%|████████▉ | 8/9 [25:52<03:15, 195.08s/it]

✅ Little Nightmares (en) — 3 мин 28 сек
🚀 Суммаризация: Among Us | Язык: en | Отзывов: 1000



Обработка игр: 100%|██████████| 9/9 [29:06<00:00, 194.01s/it]

✅ Among Us (en) — 3 мин 13 сек


In [ ]:
import os

print(os.listdir("/kaggle/input/reviews"))

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

# Путь к распакованной модели
model_path = '/kaggle/input/modelmbart'

# Загружаем токенизатор и модель
tokenizer = MBart50Tokenizer.from_pretrained(model_path)
model = MBartForConditionalGeneration.from_pretrained(model_path)

print("Модель и токенизатор успешно загружены!")


In [ ]:
#Вроде имба(отредаченная)
import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import langid
import re
import torch
from tqdm import tqdm
import random

# === Настройки ===
csv_path = "/kaggle/input/reviews/reviews_test1.csv"
game_name = "Lethal Company"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_reviews = 1000
batch_size = 30
num_beams = 5
temperature = 0.7  # Параметр для генерации более разнообразных текстов

# === Загрузка модели Flan-T5 Large ===
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
print(f"Модель Flan-T5 Large успешно загружена! Устройство: {device}")

# === Вспомогательные функции ===
def is_valid_review(text):
    if not text or len(text.strip()) < 20:
        return False
    if len(re.findall(r'\w+', text)) < 5:
        return False
    if re.fullmatch(r"[\W\d_]+", text):
        return False
    if re.search(r'---\s*\w+\s*---', text):
        return False
    if re.search(r'\b(ong|peak|10/10|best game|love it|lol|omg|hilarious|funny|memes)\b', text.lower()) and len(text.strip()) < 50:
        return False
    return True

def clean_summary(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'---.*?---', '', text)
    text = re.sub(r'[–—]{2,}', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def split_reviews_by_lang(reviews, allowed_langs=('en',)):
    lang_groups = defaultdict(list)
    for review in reviews:
        if is_valid_review(review):
            lang, _ = langid.classify(review)
            if lang in allowed_langs:
                lang_groups[lang].append(review.strip())
    return lang_groups

def summarize_text(text, max_input_len=2048, max_output_len=300):
    prompt = (
        "Summarize the following user reviews of a video game in a coherent paragraph. "
        "Highlight the gameplay, atmosphere, performance, and overall experience. "
        "Avoid repetition, usernames, or jokes:\n\n"
        + text
    )
    inputs = tokenizer(prompt, return_tensors="pt", max_length=max_input_len, truncation=True).to(device)
    summary_ids = model.generate(
        **inputs,
        max_length=max_output_len,
        min_length=100,
        do_sample=True,
        temperature=0.7,
        top_k=30,
        top_p=0.9,
        repetition_penalty=1.5,
        num_beams=1,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_reviews(reviews, batch_size=batch_size):
    partial_summaries = []
    for i in tqdm(range(0, len(reviews), batch_size), desc="Обработка отзывов"):
        batch = reviews[i:i+batch_size]
        text = " ".join(batch)
        summary = summarize_text(text)
        partial_summaries.append(summary)

    combined_text = " ".join(partial_summaries)
    return summarize_text(combined_text, max_input_len=2048, max_output_len=300)

def sample_reviews(reviews, n=max_reviews):
    return random.sample(reviews, n) if len(reviews) > n else reviews

# === Загрузка и суммаризация ===
df = pd.read_csv(csv_path, sep=';')
game_reviews = df[df["game_title"] == game_name]["review_text"].dropna().tolist()

lang_groups = split_reviews_by_lang(game_reviews)

for lang, reviews in lang_groups.items():
    if len(reviews) < 10:
        print(f"[{lang}] Мало отзывов — пропуск")
        continue
    reviews = sample_reviews(reviews)
    print(f"Суммаризация для игры: {game_name}, язык: {lang}, отзывов: {len(reviews)}")
    summary = summarize_reviews(reviews)
    summary = clean_summary(summary)
    print(f"\n📌 Итоговая суммаризация ({lang}):\n{summary}\n")


In [ ]:
#Вроде имба(отредаченная)
import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import langid
import re
import torch
from tqdm import tqdm
import random

# === Настройки ===
csv_path = "/kaggle/input/reviews/reviews_test1.csv"
game_name = "Assassin's Creed Mirage"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_reviews = 1000
batch_size = 30
num_beams = 5
temperature = 0.7  # Параметр для генерации более разнообразных текстов

# === Загрузка модели Flan-T5 Large ===
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
print(f"Модель Flan-T5 Large успешно загружена! Устройство: {device}")

# === Вспомогательные функции ===
def is_valid_review(text):
    if not text or len(text.strip()) < 20:
        return False
    if len(re.findall(r'\w+', text)) < 5:
        return False
    if re.fullmatch(r"[\W\d_]+", text):
        return False
    if re.search(r'---\s*\w+\s*---', text):
        return False
    if re.search(r'\b(ong|peak|10/10|best game|love it|lol|omg|hilarious|funny|memes)\b', text.lower()) and len(text.strip()) < 50:
        return False
    return True

def clean_summary(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'---.*?---', '', text)
    text = re.sub(r'[–—]{2,}', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def split_reviews_by_lang(reviews, allowed_langs=('en',)):
    lang_groups = defaultdict(list)
    for review in reviews:
        if is_valid_review(review):
            lang, _ = langid.classify(review)
            if lang in allowed_langs:
                lang_groups[lang].append(review.strip())
    return lang_groups

def summarize_text(text, max_input_len=2048, max_output_len=300):
    prompt = (
        "Summarize the following user reviews of a video game in a coherent paragraph. "
        "Highlight the gameplay, atmosphere, performance, and overall experience. "
        "Avoid repetition, usernames, or jokes:\n\n"
        + text
    )
    inputs = tokenizer(prompt, return_tensors="pt", max_length=max_input_len, truncation=True).to(device)
    summary_ids = model.generate(
        **inputs,
        max_length=max_output_len,
        min_length=100,
        do_sample=True,
        temperature=0.7,
        top_k=30,
        top_p=0.9,
        repetition_penalty=1.5,
        num_beams=1,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def summarize_reviews(reviews, batch_size=batch_size):
    partial_summaries = []
    for i in tqdm(range(0, len(reviews), batch_size), desc="Обработка отзывов"):
        batch = reviews[i:i+batch_size]
        text = " ".join(batch)
        summary = summarize_text(text)
        partial_summaries.append(summary)

    combined_text = " ".join(partial_summaries)
    return summarize_text(combined_text, max_input_len=2048, max_output_len=300)

def sample_reviews(reviews, n=max_reviews):
    return random.sample(reviews, n) if len(reviews) > n else reviews

# === Загрузка и суммаризация ===
df = pd.read_csv(csv_path, sep=';')
game_reviews = df[df["game_title"] == game_name]["review_text"].dropna().tolist()

lang_groups = split_reviews_by_lang(game_reviews)

for lang, reviews in lang_groups.items():
    if len(reviews) < 10:
        print(f"[{lang}] Мало отзывов — пропуск")
        continue
    reviews = sample_reviews(reviews)
    print(f"Суммаризация для игры: {game_name}, язык: {lang}, отзывов: {len(reviews)}")
    summary = summarize_reviews(reviews)
    summary = clean_summary(summary)
    print(f"\n📌 Итоговая суммаризация ({lang}):\n{summary}\n")
